In [1]:
#Load dependencies
import pandas as pd
import numpy as np
from statistics import mean
import statistics

In [2]:
#Load csv file
fantasy_df = pd.read_csv('Resources/purchase_data.csv')
fantasy_df.head(10)

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44
5,5,Yalae81,22,Male,81,Dreamkiss,3.61
6,6,Itheria73,36,Male,169,"Interrogator, Blood Blade of the Queen",2.18
7,7,Iskjaskst81,20,Male,162,Abyssal Shard,2.67
8,8,Undjask33,22,Male,21,Souleater,1.10
9,9,Chanosian48,35,Other / Non-Disclosed,136,Ghastly Adamantite Protector,3.58


In [3]:
#Number of players
fantasy_df.count()
fantasy_df['SN'].unique()
num_of_players = len(fantasy_df['SN'].unique())
num_of_players

576

In [4]:
#number of rows in dataset
len(fantasy_df)

780

In [5]:
#number of unique items
#method 1
num_of_items = len(fantasy_df['Item ID'].unique())
num_of_items

183

In [119]:
#number of unique items
#method 2
inventory_df = pd.DataFrame(fantasy_df[['Item ID','Item Name', 'Price']].drop_duplicates(['Item ID']))
num_of_items = len(inventory_df)
num_of_items

183

In [19]:
#average price
prices = list(inventory_df['Price'])
#prices = list(fantasy_df['Price'].unique())
avg_price = round(mean(prices),2)
avg_price
#len(prices)

3.04

In [8]:
#total number of purchases
tot_num_of_purch = len(fantasy_df['Item ID'])
tot_num_of_purch

780

In [9]:
#total revenue
tot_rev = fantasy_df['Price'].sum()
tot_rev

2379.77

## Gender Demographics

In [20]:
#fantasy_df['Gender'].value_counts()
gender_demo = fantasy_df.drop_duplicates(['SN'])
len(gender_demo)

576

In [11]:
gender_summary = pd.DataFrame(gender_demo['Gender'].value_counts())
gender_summary.reset_index()
gender_summary = gender_summary.rename(columns={'Gender': 'Counts'})
gender_summary

,Counts
Male,484
Female,81
Other / Non-Disclosed,11


In [12]:
gender_summary['Percentage(%)'] = round((gender_summary['Counts']/num_of_players)*100,2)
gender_summary

,Counts,Percentage(%)
Male,484,84.03
Female,81,14.06
Other / Non-Disclosed,11,1.91


In [ ]:
#not used
purchase_analysis = fantasy_df.groupby('Gender').count()
df1 = purchase_analysis[['Item ID']]
df1 = df1.rename(columns={'Item ID': 'Purchase Count'})
df1

## Purchase Analysis (Gender)

In [71]:
#selected method for purchase analysis summary
purchase_analysis = fantasy_df.groupby('Gender', as_index=False).agg({'Item ID': 'count', 
                                                                      'Price': ['sum', lambda x: round(x.unique().mean(),2)]})
#purchase_analysis = purchase_analysis.columns.set_levels(['b1','c1','f1'],level=1,inplace=True)
purchase_analysis.columns = purchase_analysis.columns.droplevel(level=1)
purchase_analysis

,Gender,Item ID,Price,Price
0,Female,113,361.94,3.11
1,Male,652,1967.64,3.04
2,Other / Non-Disclosed,15,50.19,3.31


In [57]:
#rename columns
purchase_analysis.columns = ['Gender', 'Item Count', 'Tot Revenue', 'Average Price']
purchase_analysis

,Gender,Item Count,Tot Revenue,Average Price
0,Female,113,361.94,3.11
1,Male,652,1967.64,3.04
2,Other / Non-Disclosed,15,50.19,3.31


In [73]:
#average purchase by person by gender
purchase_analysis2 = fantasy_df.groupby(['Gender', 'SN'], as_index=False).agg({'Item ID': 'count', 
                                                                      'Price': 'sum'})
purchase_analysis2
#purchase_analysis2 = purchase_analysis2.groupby('Gender', as_index=False).agg({'Price': 'mean'})
purchase_analysis2 = purchase_analysis2.groupby('Gender').agg(
    Avg_Purchase_Tot = pd.NamedAgg(column = 'Price', aggfunc= lambda x: round(mean(x),2))
)
purchase_analysis2 = purchase_analysis2.reset_index()
purchase_analysis2

,Gender,Avg_Purchase_Tot
0,Female,4.47
1,Male,4.07
2,Other / Non-Disclosed,4.56


In [23]:
#final summary
purchase_analysis_gender = pd.merge(purchase_analysis,purchase_analysis2)
purchase_analysis_gender

,Gender,Item ID,Price,Price,Avg_Purchase_Tot
0,Female,113,361.94,3.11,4.47
1,Male,652,1967.64,3.04,4.07
2,Other / Non-Disclosed,15,50.19,3.31,4.56


In [ ]:
#not used
fantasy_df.groupby('Gender').agg(
    Purchase_Count = ('Item ID', 'count'),
    Total_Revenue = ('Price', sum),
    #Average_Price = ("Price", lambda x: sum(x))
)

In [ ]:
#not used
fantasy_df.groupby('Gender').agg(
    Purchase_Count = pd.NamedAgg(column = "Item ID", aggfunc = 'count'),
    Average_Price = pd.NamedAgg(column = "Price", aggfunc = lambda x: round(x.unique().mean(),2)),
    #Total_Revenue = pd.NamedAgg(column = "Price", aggfunc = sum)
    
)

## Age Demographics

In [24]:
print(fantasy_df['Age'].max())
print(fantasy_df['Age'].min())

45
7


In [25]:
bins = [0, 10, 14, 18, 22, 26, 30, 34, 38, 42, 46, 50]
labels = ["<10", "10-14", "15-18", "19-22", "23-26", "27-30", "31-34", "35-38", "39-42", "43-46", "47-50"]
fantasy_df['Age Group'] = pd.cut(fantasy_df['Age'], bins, labels=labels)

In [26]:
fantasy_df.head(10)

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price,Age Group
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53,19-22
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56,39-42
2,2,Ithergue48,24,Male,92,Final Critic,4.88,23-26
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27,23-26
4,4,Iskosia90,23,Male,131,Fury,1.44,23-26
5,5,Yalae81,22,Male,81,Dreamkiss,3.61,19-22
6,6,Itheria73,36,Male,169,"Interrogator, Blood Blade of the Queen",2.18,35-38
7,7,Iskjaskst81,20,Male,162,Abyssal Shard,2.67,19-22
8,8,Undjask33,22,Male,21,Souleater,1.10,19-22
9,9,Chanosian48,35,Other / Non-Disclosed,136,Ghastly Adamantite Protector,3.58,35-38


In [90]:
#selected method for purchase analysis summary
age_demo = fantasy_df.groupby('Age Group', as_index=False).agg({'Item ID': 'count', 
                                                                      'Price': ['sum', lambda x: round(x.unique().mean(),2)]})
#age_demo = age_demo.columns.set_levels(['b1','c1','f1'],level=1,inplace=True)
age_demo.columns = age_demo.columns.droplevel(level=1)
#age_demo.loc[columns[2]].fillna(0)
age_demo['Price'].fillna(0)
#rename columns
age_demo.columns = ['Age Group', 'Item Count', 'Tot Revenue', 'Average Price']
age_demo['Average Price'] = age_demo['Average Price'].fillna(0)
#age_demo.dtypes
age_demo

,Age Group,Item Count,Tot Revenue,Average Price
0,<10,32,108.96,3.41
1,10-14,19,50.95,2.77
2,15-18,113,342.91,2.99
3,19-22,254,771.89,3.03
4,23-26,207,634.24,3.12
5,27-30,63,181.23,2.88
6,31-34,38,103.68,2.75
7,35-38,35,124.35,3.50
8,39-42,15,50.50,3.37
9,43-46,4,11.06,2.76


In [65]:
fantasy_df

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price,Age Group
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53,19-22
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56,39-42
2,2,Ithergue48,24,Male,92,Final Critic,4.88,23-26
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27,23-26
4,4,Iskosia90,23,Male,131,Fury,1.44,23-26
...,...,...,...,...,...,...,...,...
775,775,Aethedru70,21,Female,60,Wolf,3.54,19-22
776,776,Iral74,21,Male,164,Exiled Doomblade,1.63,19-22
777,777,Yathecal72,20,Male,67,"Celeste, Incarnation of the Corrupted",3.46,19-22
778,778,Sisur91,7,Male,101,Final Critic,4.19,<10


In [88]:
#average purchase by person by age group
age_demo2 = fantasy_df.groupby(['Age Group', 'SN']).agg({'Item ID': 'count', 
                                                         'Price': 'sum'})
age_demo2 = age_demo2.dropna()
#age_demo2 = age_demo2.groupby('Gender', as_index=False).agg({'Price': 'mean'})
#age_demo2 = age_demo2.groupby('Age Group').agg(
#    Avg_Purchase_Tot = pd.NamedAgg(column = 'Price', aggfunc= lambda x: round(mean(x),2))
#)
age_demo2 = age_demo2.reset_index()
#age_demo2.groupby('Age Group').count()
age_demo2 = age_demo2.groupby('Age Group').agg(
    Avg_Purchase_Tot = pd.NamedAgg(column = 'Price', aggfunc= lambda x: round(mean(x),2))
)
age_demo2 = age_demo2.reset_index()
age_demo2

,Age Group,Avg_Purchase_Tot
0,10-14,3.40
1,15-18,3.81
2,19-22,4.34
3,23-26,4.20
4,27-30,3.78
5,31-34,3.84
6,35-38,4.97
7,39-42,3.61
8,43-46,2.77
9,<10,4.54


In [93]:
#final summary
purchase_analysis_age = pd.merge(age_demo,age_demo2, how='outer')
purchase_analysis_age['Avg_Purchase_Tot'] = purchase_analysis_age['Avg_Purchase_Tot'].fillna(0)
purchase_analysis_age

,Age Group,Item Count,Tot Revenue,Average Price,Avg_Purchase_Tot
0,<10,32,108.96,3.41,4.54
1,10-14,19,50.95,2.77,3.40
2,15-18,113,342.91,2.99,3.81
3,19-22,254,771.89,3.03,4.34
4,23-26,207,634.24,3.12,4.20
5,27-30,63,181.23,2.88,3.78
6,31-34,38,103.68,2.75,3.84
7,35-38,35,124.35,3.50,4.97
8,39-42,15,50.50,3.37,3.61
9,43-46,4,11.06,2.76,2.77


## Top Spenders

In [127]:
#Top five spenders
grouped_by_players = fantasy_df.groupby('SN', as_index=False).agg({'Purchase ID': 'count', 
                                                                      'Price': ['sum', lambda x: round(x.unique().mean(),2)]})
grouped_by_players.columns = grouped_by_players.columns.droplevel(level=1)
grouped_by_players.columns = ['SN', 'Item Count', 'Tot Revenue', 'Average Price']
players_sort_by_revenue = grouped_by_players.sort_values(by=['Tot Revenue'], ascending=False)
top_five_players = players_sort_by_revenue.iloc[0:5]
top_five_players = top_five_players.reset_index(drop=True)
top_five_players

,SN,Item Count,Tot Revenue,Average Price
0,Lisosia93,5,18.96,3.79
1,Idastidru52,4,15.45,3.86
2,Chamjask73,3,13.83,4.61
3,Iral74,4,13.62,3.40
4,Iskadarya95,3,13.10,4.37


## Most Popular Items

In [131]:
# Most Popular Items
grouped_by_items = fantasy_df.groupby('Item ID', as_index=False).agg({'Purchase ID': 'count', 
                                                                      'Price': 'sum'})
#grouped_by_items.columns = grouped_by_items.columns.droplevel(level=1)
grouped_by_items.columns = ['Item ID', 'Purchase Count', 'Tot Revenue']
grouped_by_items = grouped_by_items.merge(inventory_df)
grouped_by_items
items_sort_by_count= grouped_by_items.sort_values(by=['Purchase Count'], ascending=False)
top_five_items_count = items_sort_by_count.iloc[0:5]
top_five_items_count = top_five_items_count.reset_index(drop=True)
top_five_items_count
#items_sort_by_count.head(15)
#ask instructor how to do rankings with equal spots

,Item ID,Purchase Count,Tot Revenue,Item Name,Price
0,178,12,50.76,"Oathbreaker, Last Hope of the Breaking Storm",4.23
1,145,9,41.22,Fiery Glass Crusader,4.58
2,108,9,31.77,"Extraction, Quickblade Of Trembling Hands",3.53
3,82,9,44.10,Nirvana,4.90
4,19,8,8.16,"Pursuit, Cudgel of Necromancy",1.02


## Most Profitable Items

In [132]:
# Most Profitable Items
items_sort_by_revenue= grouped_by_items.sort_values(by=['Tot Revenue'], ascending=False)
top_five_items_revenue = items_sort_by_revenue.iloc[0:5]
top_five_items_revenue = top_five_items_revenue.reset_index(drop=True)
top_five_items_revenue
#items_sort_by_count.head(15)
#ask instructor how to do rankings with equal spots

,Item ID,Purchase Count,Tot Revenue,Item Name,Price
0,178,12,50.76,"Oathbreaker, Last Hope of the Breaking Storm",4.23
1,82,9,44.10,Nirvana,4.90
2,145,9,41.22,Fiery Glass Crusader,4.58
3,92,8,39.04,Final Critic,4.88
4,103,8,34.80,Singed Scalpel,4.35
